# Synthetic Optical Flow from Fused Lidar


In [1]:
import sys
sys.path.append('/opt/psegs')

import numpy as np

from psegs.exp.fused_lidar_flow import FusedLidarCloudTableBase
from psegs.exp.fused_lidar_flow import TaskLidarCuboidCameraDFFactory

import IPython.display
import PIL.Image


## General Notebook Utilities
    
def imshow(x):
    IPython.display.display(PIL.Image.fromarray(x))

def show_html(x):
    from IPython.core.display import display, HTML
    display(HTML(x))

## SemanticKITTI

In [2]:
from psegs.exp.semantic_kitti import SemanticKITTISDTable

class SemanticKITTILCCDFFactory(TaskLidarCuboidCameraDFFactory):
    
    SRC_SD_TABLE = SemanticKITTISDTable
    
    @classmethod
    def build_df_for_segment(cls, spark, segment_uri):
        seg_rdd = cls.SRC_SD_TABLE.get_segment_datum_rdd(spark, segment_uri)
        
        def to_task_row(scan_id_iter_sds):
            scan_id, iter_sds = scan_id_iter_sds
            camera_images = []
            point_clouds = []
            for sd in iter_sds:
                if sd.camera_image is not None:
                    camera_images.append(sd)
                elif sd.point_cloud is not None:
                    point_clouds.append(sd)
            
            from pyspark import Row
            r = Row(
                    task_id=int(scan_id),
                    pc_sds=point_clouds,
                    cuboids_sds=[], # SemanticKITTI has no cuboids
                    ci_sds=camera_images) 
            from oarphpy.spark import RowAdapter
            return RowAdapter.to_row(r)
            
        grouped = seg_rdd.groupBy(lambda sd: sd.uri.extra['semantic_kitti.scan_id'])
        row_rdd = grouped.map(to_task_row)

        df = spark.createDataFrame(row_rdd, schema=cls.table_schema())
        df = df.persist()
        return df

class SemanticKITTIFusedWorldCloudTable(FusedLidarCloudTableBase):
    TASK_DF_FACTORY = SemanticKITTILCCDFFactory

    # SemanticKITTI has no cuboids, so we skip this step.
    HAS_OBJ_CLOUDS = False


## KITTI-360

In [3]:
from psegs.datasets.kitti_360 import KITTI360SDTable
class KITTI360OurFusedClouds(KITTI360SDTable):
    INCLUDE_FISHEYES = False
    INCLUDE_FUSED_CLOUDS = False  # Use our own fused clouds

class KITTI360OurFusedWorldCloudTable(FusedLidarCloudTableBase):
    SRC_SD_TABLE = KITTI360OurFusedClouds
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        spark.catalog.dropTempView('culi_tasks_df')
        print('Building tasks table for %s ...' % segment_uri.segment_id)
        spark.sql("""
          CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
          SELECT 
              CONCAT(uri.segment_id, '.', uri.extra.`kitti-360.frame_id`) AS task_id,
              FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
              COLLECT_LIST(point_cloud) AS point_clouds
          FROM datums
          WHERE 
              uri.topic LIKE '%cuboid%' OR uri.topic LIKE '%lidar%'
          GROUP BY task_id
        """)
        
        
        # TODO! for lidar and camera image!
        #         both_have_ego_pose = (
        #             ci1.extra.get('kitti-360.has-valid-ego-pose') and
        #             ci2.extra.get('kitti-360.has-valid-ego-pose'))
        
        tasks_df = spark.sql('SELECT * FROM culi_tasks_df')
        print('... done.')
        return tasks_df.rdd


## NuScenes

In [4]:
# !pip3 install nuscenes-devkit==1.1.2
from psegs.datasets.nuscenes import NuscStampedDatumTableBase, NuscStampedDatumTableLabelsAllFrames
class NuscFusedWorldCloudTableBase(FusedLidarCloudTableBase):
    #SRC_SD_TABLE = NuscStampedDatumTableBase -- Subclass must pick a NuScenes configuration!
    
    SPLITS = ['train_detect', 'train_track']
    
    @classmethod
    def _filter_ego_vehicle(cls, cloud_ego):
        # Note: NuScenes authors have already corrected clouds for ego motion:
        # https://github.com/nutonomy/nuscenes-devkit/issues/481#issuecomment-716250423
        # But have not filtered out ego self-returns
        cloud_ego = cloud_ego[np.where(  ~(
                        (cloud_ego[:, 0] <= 1.5) & (cloud_ego[:, 0] >= -1.5) &  # Nusc lidar +x is +right
                        (cloud_ego[:, 1] <= 2.5) & (cloud_ego[:, 0] >= -2.5) &  # Nusc lidar +y is +forward
                        (cloud_ego[:, 1] <= 1.5) & (cloud_ego[:, 0] >= -1.5)    # Nusc lidar +z is +up
        ))]
        return cloud_ego
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        spark.catalog.dropTempView('culi_tasks_df')
        print('Building tasks table for %s ...' % segment_uri.segment_id)
        if cls.SRC_SD_TABLE.LABELS_KEYFRAMES_ONLY:
            # For Nusc: group by nuscenes-sample-token WITH KEYFRAMES
            spark.sql("""
              CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
              SELECT 
                  uri.extra.`nuscenes-sample-token` AS task_id,
                  FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
                  COLLECT_LIST(point_cloud) AS point_clouds
              FROM datums
              WHERE 
                uri.extra.`nuscenes-is-keyframe` = 'True' AND (
                  uri.extra['nuscenes-label-channel'] is NULL OR 
                  uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
                ) AND (
                  uri.topic LIKE '%cuboid%' OR
                  uri.topic LIKE '%lidar%'
                )
              GROUP BY task_id
            """)
        else:
            # For Nusc: group by nuscenes-sample-token WITH ALL FRAMES
            spark.sql("""
              CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
              SELECT 
                  CONCAT(uri.segment_id, '.', uri.timestamp) AS task_id,
                  FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
                  COLLECT_LIST(point_cloud) AS point_clouds
              FROM datums
              WHERE 
                (
                  uri.extra['nuscenes-label-channel'] is NULL OR 
                  uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
                ) AND (
                  uri.topic LIKE '%cuboid%' OR
                  uri.topic LIKE '%lidar%'
                )
              GROUP BY task_id
              HAVING SIZE(cuboids) > 0 AND SIZE(point_clouds) > 0
            """)
        
        tasks_df = spark.sql('SELECT * FROM culi_tasks_df')
        print('... done.')
        return tasks_df.rdd

class NuscFusedWorldCloudKeyframesOnlyTable(NuscFusedWorldCloudTableBase):
    SRC_SD_TABLE = NuscStampedDatumTableBase

class NuscFusedWorldCloudAllFramesTable(NuscFusedWorldCloudTableBase):
    SRC_SD_TABLE = NuscStampedDatumTableLabelsAllFrames
    

## Start Spark

In [5]:
from psegs.spark import NBSpark
spark = NBSpark.getOrCreate()

2021-02-19 21:37:23,149	oarph 2168770 : Using source root /opt/psegs/psegs 
2021-02-19 21:37:23,150	oarph 2168770 : Using source root /opt/psegs 
2021-02-19 21:37:23,247	oarph 2168770 : Generating egg to /tmp/tmphise9bk1_oarphpy_eggbuild ...
2021-02-19 21:37:23,337	oarph 2168770 : ... done.  Egg at /tmp/tmphise9bk1_oarphpy_eggbuild/psegs-0.0.0-py3.8.egg


## Build Fused Lidar Assets

```
docker --context default run -it --name=potree_viewer --rm --net=host -v `pwd`:/shared  jonazpiazu/potree
```

In [6]:
# T = KITTI360OurFusedWorldCloudTable
# rdds = T._create_datum_rdds(spark)
# print([r.count() for r in rdds])

# seg_uris = T.get_all_segment_uris()
# samp = T.get_sample(seg_uris[0], spark=spark)


In [7]:
# print([lc.sensor_name for lc in samp.lidar_clouds][:10])
# c = samp.lidar_clouds[0]#[lc for lc in samp.lidar_clouds if lc.sensor_name == '11002'][0]
# print(c.get_cloud().shape)
# imshow(c.get_bev_debug_image(x_bounds_meters=None, y_bounds_meters=None))
# imshow(c.get_front_rv_debug_image(y_bounds_meters=None, z_bounds_meters=None))

## Compute Candidate Optical Flow Pairs

## Render Optical Flow

In [ ]:
from psegs.exp.fused_lidar_flow import OpticalFlowRenderBase

class SemanticKITTIOFlowRenderer(OpticalFlowRenderBase):
    FUSED_LIDAR_SD_TABLE = SemanticKITTIFusedWorldCloudTable
    MAX_TASKS_PER_SEGMENT = 50

R = SemanticKITTIOFlowRenderer
seg_uris = R.FUSED_LIDAR_SD_TABLE.get_all_segment_uris()
R.build(spark=spark, only_segments=[seg_uris[0]])

2021-02-19 21:37:26,682	ps   2168770 : Found Sequence 00 with 4541 scans
2021-02-19 21:37:26,683	ps   2168770 : Found Sequence 01 with 1101 scans
2021-02-19 21:37:26,686	ps   2168770 : Found Sequence 02 with 4661 scans
2021-02-19 21:37:26,687	ps   2168770 : Found Sequence 03 with 801 scans
2021-02-19 21:37:26,688	ps   2168770 : Found Sequence 05 with 2761 scans
2021-02-19 21:37:26,689	ps   2168770 : Found Sequence 06 with 1101 scans
2021-02-19 21:37:26,690	ps   2168770 : Found Sequence 07 with 1101 scans
2021-02-19 21:37:26,691	ps   2168770 : Found Sequence 09 with 1591 scans
2021-02-19 21:37:26,692	ps   2168770 : Found Sequence 10 with 1201 scans
2021-02-19 21:37:26,692	ps   2168770 : Found 18859 total scans
2021-02-19 21:37:26,693	ps   2168770 : Filtering to only 1 segments
2021-02-19 21:37:26,694	ps   2168770 : Finding scans for sequence 00 with no moving points ...
2021-02-19 21:37:33,102	ps   2168770 : ... sequence 00 has 3097 scans with no movers.
2021-02-19 21:37:47,578	ps   216

num tasks 3097
restrict to 50
running map.. count
